In [1]:
import sys 

sys.path.append('../input/flashlib/Flash-trainer')

In [2]:
!pip install neptune-client
!pip install -U segmentation-models-pytorch

     |████████████████████████████████| 278 kB 285 kB/s 
     |████████████████████████████████| 67 kB 2.4 MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.13.1-py2.py3-none-any.whl size=482645 sha256=8d2788e8d6b784aa00d4d62e843690b43649f73f60b8a2335607169163da32db
  Stored in directory: /root/.cache/pip/wheels/6a/fd/22/31d584fb3011b4d5dc428d879e4e2a0708869dc80e33cd3c7f
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7-py3-none-any.whl size=18149 sha256=00fe1a7de3f57181f2af1933b9b53bf82a5121a4a5f27403ced07e3df39665a1
  Stored in directory: /root/.cache/pip/wheels/f3/1d/9f/2a74caecb81b8beb9a4fbe1754203d4b7cf42ef5d39e0d2311
Successfully built neptune-client strict-rfc3339
     |████████████████████████████████| 88 kB 364 kB/s 
     |████████████████████████████████| 376 kB 585 kB/s 
     |████████████████████████████████| 58 kB 3.3 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=20314b

In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import segmentation_models_pytorch as smp

from sklearn import metrics
from sklearn import model_selection

import torch 
from torch import nn 
from torch.utils.data import Dataset
import torch.nn.functional as F

import cv2
import os 

import Flash 
from Flash.model import Model 
from Flash.activations import Mish, Swish
from Flash.optimizers import Ranger, MADGRAD, Lookahead 

import albumentations as A 
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

In [4]:
def run_length_decode(rle, height=1024, width=1024, fill_value=1):
    component = np.zeros((height, width), np.float32)
    component = component.reshape(-1)
    rle = np.array([int(s) for s in rle.strip().split(' ')])
    rle = rle.reshape(-1, 2)
    start = 0
    for index, length in rle:
        start = start+index
        end = start+length
        component[start: end] = fill_value
        start = end
    component = component.reshape(width, height).T
    return component

In [5]:
params = {
    'DATA_DIR' : '../input/siim-png-images/train_png/',
    'TRAIN_CSV' : '../input/ps-folds/train_folds.csv',

    'SEED' : 42,
    'FOLD' : 1,
    'IMG_SIZE' : 384,

    'EPOCHS' : 12,
    'BATCH_SIZE' : 16,

    'NUM_WORKERS' : 4,
    'DEVICE' : 'cuda',
    'OPTIMIZER' : 'ranger',
    'CRIT' : 'focal_loss + dice',
    'SCHEDULER' : 'cosine',

    'OUTPUT_DIM' : 1,
    'ENCODER' : 'timm-efficientnet-b0',
    'DEVICE' : 'cuda',

    'LR' : 0.0003,
    'T_MAX' : 12,
    'ETA_MIN' : 1e-6,

    'FP16' : True,

    'SCHEDULER_PARAMS' : {
        "lr_start": 1e-4,
        "lr_max": 1e-4 * 32,
        "lr_min": 1e-6,
        "lr_ramp_ep": 6,
        "lr_sus_ep": 0,
        "lr_decay": 0.75,
    }
}

In [6]:
df = pd.read_csv(params['TRAIN_CSV'])
df.head()

,ImageId,EncodedPixels,kfold
0,1.2.276.0.7230010.3.1.4.8323329.10784.15178752...,-1,0
1,1.2.276.0.7230010.3.1.4.8323329.14503.15178752...,-1,0
2,1.2.276.0.7230010.3.1.4.8323329.3100.151787517...,-1,0
3,1.2.276.0.7230010.3.1.4.8323329.31831.15178751...,405665 2 1021 3 1020 4 1019 5 1018 5 1018 6 1...,0
4,1.2.276.0.7230010.3.1.4.8323329.11993.15178752...,-1,0


In [7]:
class PneumothoraxDataset(Dataset):
    
    def __init__(self, df, preprocessing_fun = None, augmentations = None):
        self.df = df
        self.augmentations = augmentations
        self.preprocessing_fun = preprocessing_fun
        self.channel_first = True
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        
        row = self.df.iloc[idx]
        
        image_id = row['ImageId']
        enc_pix = [row[' EncodedPixels']]
        
        img_path = os.path.join(params['DATA_DIR'], image_id + ".png")
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        mask = np.zeros(shape=(1024,1024))
        if enc_pix[0] != ' -1':
            for rle in enc_pix:
                mask += run_length_decode(rle)
        mask = (mask >= 1).astype('float32')
        mask = np.expand_dims(mask, axis=-1)
        
        if self.augmentations:
            augmented = self.augmentations(image=img, mask=mask)
            img = augmented['image']
            mask = augmented['mask']
            
        if self.preprocessing_fun:
            img = self.preprocessing_fun(img)
            
        if self.channel_first:
            img = np.transpose(img, (2, 0, 1)).astype(np.float32)
            mask = np.transpose(mask, (2, 0, 1)).astype(np.float32)
            
        return {
            'images' : torch.Tensor(img),
            'masks' : torch.Tensor(mask)
        }

In [8]:
train_augs = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=10, p=0.5),
    A.OneOf([A.RandomGamma(gamma_limit=(90,110)),
             A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)], p=0.5),
    A.Cutout(max_h_size=int(params['IMG_SIZE'] * 0.05), max_w_size=int(params['IMG_SIZE'] * 0.05), num_holes=10, p=0.5),
    A.Resize(params['IMG_SIZE'], params['IMG_SIZE'])
])

valid_augs = A.Compose([
    A.Resize(params['IMG_SIZE'], params['IMG_SIZE'])
])

prep_fun = smp.encoders.get_preprocessing_fn(
    params['ENCODER'],
    'imagenet'
)

/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:691: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,


In [9]:
def dice_loss(input, target):
    input = torch.sigmoid(input)
    smooth = 1.0
    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    return ((2.0 * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth))


class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma

    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))
        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()
        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        return loss.mean()


class MixedLoss(nn.Module):
    def __init__(self, alpha, gamma):
        super().__init__()
        self.alpha = alpha
        self.focal = FocalLoss(gamma)

    def forward(self, input, target):
        loss = self.alpha*self.focal(input, target) - torch.log(dice_loss(input, target))
        return loss.mean()

In [10]:
class PneumothoraxModel(Model):
    
    def __init__(self, pretrained = True):
        super(PneumothoraxModel, self).__init__()
        
        self.backbone = smp.Unet(
            encoder_name = params['ENCODER'],
            encoder_weights = 'imagenet',
            in_channels = 3,
            classes = 1,
            activation = None
        )
        
    def configure_optimizer(self):
        return Ranger(self.parameters(), lr = params['LR'])
    
    def configure_scheduler(self):
        return torch.optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer,
            T_max = params['T_MAX'],
            eta_min = params['ETA_MIN']
        )
    
    def forward(self, images, masks):
        
        logits = self.backbone(images)
        
        if masks != None:
            return logits, MixedLoss(alpha = 10.0, gamma = 2.0)(logits, masks), None
        
        return logits

In [11]:
model = PneumothoraxModel()
model.to(params['DEVICE']);

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b0_aa-827b6e33.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b0_aa-827b6e33.pth


  0%|          | 0.00/20.4M [00:00<?, ?B/s]

In [12]:
df_train = df[df['kfold'] != params['FOLD']]
df_valid = df[df['kfold'] == params['FOLD']]

trainset = PneumothoraxDataset(df_train, prep_fun, train_augs)
validset = PneumothoraxDataset(df_valid, prep_fun, valid_augs)

In [13]:
model.fit(
    trainset,
    validset,
    batch_size = params['BATCH_SIZE'],
    device = 'cuda',
    fp16 = False,
    epochs = params['EPOCHS'],
    train_shuffle = True,
    valid_shuffle = False,
    pin_memory = False,
    num_workers = 2,
    on_end_metrics = False,
    save_model_based_on_metric = False,
    save_model_based_on_loss = True,
    target_name = 'masks',
    model_path =  'Final_'+ params['ENCODER'] + '_' + str(params['FOLD']) + '.pt',
    drop_last_batch_train = True,
    logger = True,
    neptune_api_token = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIyMGIxNDQyZC00MmI4LTQyZmUtYjg5NS01ODUyNzRkZWNlMzEifQ==",
    neptune_init = 'parthdhameliya/Pneumothorax-seg',
    neptune_experiment_name = 'experiment1-'+params['ENCODER'],
    neptune_params = params
)

https://app.neptune.ai/parthdhameliya/Pneumothorax-seg/e/PNEUM-11
Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers


Epoch [VALID] 1: 100%|██████████| 145/145 [01:01<00:00,  2.36it/s, val_live_loss=3.959908]


MODEL_SAVED
train metrics : {}
valid metrics : {}
Epoch : 1 train_loss : 5.213209038571373 valid_loss : 3.9599079049866774


Epoch [VALID] 2: 100%|██████████| 145/145 [00:57<00:00,  2.51it/s, val_live_loss=2.213498]


MODEL_SAVED
train metrics : {}
valid metrics : {}
Epoch : 2 train_loss : 3.1685407791730653 valid_loss : 2.2134982109069825


...

MODEL_SAVED
train metrics : {}
valid metrics : {}
Epoch : 10 train_loss : 1.0630181012273041 valid_loss : 1.2045866175972182


Epoch [VALID] 11: 100%|██████████| 145/145 [00:58<00:00,  2.48it/s, val_live_loss=1.263517]


train metrics : {}
valid metrics : {}
Epoch : 11 train_loss : 0.9952061978234734 valid_loss : 1.2635167951213901


Epoch [VALID] 12: 100%|██████████| 145/145 [00:59<00:00,  2.44it/s, val_live_loss=1.253719]

train metrics : {}
valid metrics : {}
Epoch : 12 train_loss : 0.9704193823644328 valid_loss : 1.2537192040476306
